# Let’s embrace WebAssembly!

EuroPython 2018 - Edinburgh

Almar Klein

<img src='images/wasm.png' width=100 align=right>
<img src='images/python.png' width=100 align=right>


# What is WebAssembly?

<img src='images/wasm.png' align=right width=300>



WebAssembly == WASM

# WASM is an OPEN standard ...

<br>
<div style='display:flex; width:100%; justify-content:space-around; align-items:center;'>
<img src='images/group.png' style='height:200px; width:200px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

Collaborative effort by Mozilla, Google, Apple, Microsoft ...


# ... for executable code

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:200px; width:200px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

It's fast!

<img src='images/assembly1.png' width=600>

<img src='images/assembly2.png' width=600>


# It has a compact binary format

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:200px; width:200px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

And a human readable counterpart:

```wasm
(module
(type $print (func (param i32))
(func $main
    (i32.const 42)
    (call $print)
)
(start $main)
)
```


# It's safe

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:200px; width:200px;'>
</div>
<br>

Because browsers.

# WebAssembly is coming and it's awesome!

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:50px; width:50px;'>
<img src='images/rocket.png' style='height:50px; width:50px;'>
<img src='images/file.png' style='height:50px; width:50px;'>
<img src='images/check-square.png' style='height:50px; width:50px;'>
</div>
<br><br>

<center>
<img src='images/so-much-awesome-meme.jpg' width=500px>
</center>


# WebAssembly adoption

# Lua community

Let's write web apps in Lua!

<center><img src='images/celebrate.jpg' width=300></center>

# Rust community

Let's write apps in Rust!!

<center><img src='images/celebrate.jpg' width=300></center>



# JavaScript community


Yes, this may end our suffering!

<center><img src='images/celebrate.jpg' width=300></center>

# C++ community

Let's write web apps in C++!


<center><img src='images/celebrate.jpg' width=300px></center>





<center><img src='images/facepalm.jpg' width=300px></center>


# Python community

<br><br><br>

...


# WASM may not be obvious for Python

... Because Python is an *interpreted* language

# Three use-cases how we can embrace WASM

# Use case 1: Compile a Python interpreter



# Examples


* Pyodide: compiles CPython + numpy/pandas/matplotlib, to run in the browser
* PyPyJS
* MicroPython
* RustPython: Python interpreter written in Rust



Note: Python code is still run in a VM!

# Use case 2: Compile a subset of Python to WASM

In [ ]:
import time
from ppci import wasm

#@wasm.wasmify
def find_prime(nth):
    n = 0
    i = -1       
    while n < nth:
        i = i + 1        
        if i <= 1:
            continue  # nope
        elif i == 2:
            n = n + 1
        else:
            gotit = 1
            for j in range(2,  i//2+1):
                if i % j == 0:
                    gotit = 0
                    break
            if gotit == 1:
                n = n + 1    
    return i

In [ ]:
import time

t0 = time.perf_counter()
print(find_prime(1000), f'(took {time.perf_counter()-t0:.3f} s)')

In [ ]:
from ppci.lang.python import python_to_wasm
m = python_to_wasm(find_prime)
m.show_interface()

# Compiling Python to WASM

* Similar to Numba / Cython
* Binaries are cross-platform!

Note: 

* Needs (a lot of) work, e.g. more types
* Static compilation, only support strict subset of Python

# Use case 3: Run WASM in Python
    
... and allow that code to call into Python functions

# Rocket game

<center>
    <!-- <a href='https://thread-safe.nl/rocket/' target='new'> -->
    <a href='rocket.html' target='new'>
    <img src='images/github_rocket_wasm.png' width=900>
    </a>
</center>

# Single binary WASM file (58 KB)

<center>    
    <img src='images/github_rocket_wasm_html.png' width=600>
</center>



In [ ]:
from ppci import wasm

m = wasm.Module(open(r'wasm/rocket.wasm', 'rb'))
m

In [ ]:
m.show_interface()

<center>
<img src='images/github_rocket_wasm_js.png' width=1000>
</center>

<center>
<img src='images/rocket_in_js.png' width=800>
</center>

<center>
<img src='images/rocket_in_py.png' width=800>
</center>

In [ ]:
import time
import math

class BaseRocketGame:
    """ Simple rocket game, text based, without user input.
    """
    
    def __init__(self, game_module=m):
        # Create import dict from methods
        env = {}
        for name in dir(self):
            if name.startswith('wasm_'):
                env[name[5:]] = getattr(self, name)
        self.imports = dict(env=env)
        
        # Instantiate game module
        self.game = wasm.instantiate(game_module, self.imports, target='python')
    
    def run(self):
        """ Enter the game's main loop.
        """
        self.game.exports.resize(100, 100)
        while True:
            time.sleep(0.5)
            self.game.exports.update(0.1)
            self.game.exports.draw()
    
    def wasm_sin(self, a:float) -> float:
        return math.sin(a)
    
    def wasm_cos(self, a:float) -> float:
        return math.cos(a)
    
    def wasm_Math_atan(self, a:float) -> float:
        return math.atan(a)
    
    def wasm_clear_screen(self) -> None:
        print('clearing screen')
    
    def wasm_draw_bullet(self, x:float, y:float) -> None:
        print(f'There is a bullet at {x}, {y}')
    
    def wasm_draw_enemy(self, x:float, y:float) -> None:
        print(f'There is an enemy at {x}, {y}')
    
    def wasm_draw_particle(self, x:float, y:float, a:float) -> None:
        print(f'There is a particle at {x}, {y} angle {a}')
    
    def wasm_draw_player(self, x:float, y:float, a:float) -> None:
        print(f'The player is at {x}, {y} angle {a}')
    
    def wasm_draw_score(self, score:float) -> None:
        print(f'The score is {score}!')

# Run Rocket in Python

In [ ]:
from rocket import BaseRocketGame
game = BaseRocketGame()

In [ ]:
game.run()

# Run Rocket in Python with Qt

In [ ]:
from rocket_qt import QtRocketGame
game = QtRocketGame()

In [ ]:
game.run()

# Run Rocket in Python with prompt_toolkit

Over SSH :)

# This game does not look that hard ...

## Let's make an AI!

# AI 1

In [ ]:
#print(open('wasm/ai1.c', 'rt').read())

In [ ]:
from ppci import wasm
ai1 = wasm.Module(open('wasm/ai1.wasm', 'rb'))

In [ ]:
ai1.show_interface()

In [ ]:
from rocket_ai import AiRocketGame
game = AiRocketGame(ai1)
game.run()

# AI 2

In [ ]:
#print(open('wasm/ai2.wat', 'rt').read())

In [ ]:
ai2 = wasm.Module(open('wasm/ai2.wat', 'rt').read())

In [ ]:
ai2

In [ ]:
from rocket_ai import AiRocketGame
game = AiRocketGame(ai2)
game.run()


# Wrapping up ...

Examples build on PPCI and the hard work of Windel Bouwman

Most things that I showed are experimental

## WASM is coming, and its awesome!

<img src='images/wasm.png' width=150 align=right>

* Open, low-level, fast, compact and safe
* Already works in most browsers
* WASM can be a generic cross-platform IR



<img src='images/python.png' width=150 align=right>

## We Pythonista's should embrace it!

* Run a Python VM in the browser
* Statically compile subset of Python to fast and crossplatform code
* Bind and execute WASM modules in Python

## Thanks!